### Name : Ghosh Kushanava Amitava
### Roll No : 220123083
### Course : MA323

In [7]:
import numpy as np
import pandas as pd
from scipy.stats import norm, expon

## Q.1

In [8]:
# Linear Congruential Generator (LCG)
def lcg(a, c, m, seed, n):
    random_numbers = []
    X = seed
    for _ in range(n):
        X = (a * X + c) % m
        random_numbers.append(X / m)  # Normalizing to [0, 1]
    return np.array(random_numbers)

#### Using Non Anti-thetic Method

In [9]:
# Define the function f(U) = exp(U) in the given question
def f(U):
    return np.exp(U)

#### Using Anti-Thetic Method

In [10]:
# Define the function h(U) = (exp(U) + exp(1-U))/2 in the given question
def h(U):
    return (np.exp(U) + np.exp(1-U))/2

In [11]:
# Function to compute Monte Carlo estimate using various methods
def mce(U, F):
    return np.mean(F(U)), np.var(F(U))

In [12]:
# Function to compute Monte Carlo estimate with control variates
def mccv(M, U, f):
    Y = f(U) 
    cov_UY = np.cov(Y, U)[0, 1]
    var_U = np.var(U)
    a = cov_UY / var_U # Optimal control variate coefficient
    return np.mean(Y - a * (U - 0.5)), np.var(Y - a * (U - 0.5))


In [13]:
m = 2**31 - 1
a = 1103515245
c = 12345
seed = 42

# Values of M given in the question
M = [10, 100, 1000, 10000, 100000]

#  List to Store the results
results = []

# Actual value of the integral I = E[e^U] where U ~ U(0,1) is E[e^U] = (e - 1)
I_actual = (np.e - 1)

# Perform simulations for different values of M
for m_ in M:
    # Generate U(0, 1) random numbers using LCG
    U = lcg(a, c, m, seed, m_)
    
    # Non Anti-thetic variate Monte Carlo estimate
    I_M, V_M = mce(U, f)
    
    # Antithetic variate Monte Carlo estimate
    I_M_hat, V_M_hat = mce(U, h)
    
    # Control variates estimate
    I_M_tilda, V_M_tilda = mccv(M, U, f)
    
    # Storing the results
    results.append([m_, I_actual, I_M, V_M, I_M_hat, V_M_hat, I_M_tilda, V_M_tilda])

In [14]:
# Creating the table by using dataframe
df = pd.DataFrame(results, columns=['M', 'I_A', 'I_M', 'Var_M', 'I_M_hat', 'Var_M_hat', 'I_M_tilda', 'Var_M_tilda'])
df['% Reduction (Anti-thetic)'] = (1 - df['Var_M_hat'] / df['Var_M']) * 100
df['% Reduction (Control)'] = (1 - df['Var_M_tilda'] / df['Var_M']) * 100

df

,M,I_A,I_M,Var_M,I_M_hat,Var_M_hat,I_M_tilda,Var_M_tilda,% Reduction (Anti-thetic),% Reduction (Control)
0,10,1.718282,1.522463,0.089935,1.687215,0.001365,1.692774,0.001888,98.481720,97.900899
1,100,1.718282,1.580086,0.176621,1.707650,0.002635,1.705560,0.002421,98.507927,98.629180
2,1000,1.718282,1.712820,0.237583,1.716674,0.003718,1.716877,0.003752,98.435221,98.420589
3,10000,1.718282,1.703878,0.238044,1.717653,0.003840,1.717614,0.003856,98.386735,98.379930
4,100000,1.718282,1.716807,0.240528,1.717942,0.003890,1.717925,0.003916,98.382779,98.371789


## Q.2

In [15]:
# Basic Monte Carlo simulation for P(X > 1)
def mcb(M, U1, U2):
    Y = -np.log(1 - U1)  # exp(1) distribution from U(0,1)
    X = np.sqrt(4) * U2 + Y  # Gaussian with mean Y and variance 4
    return np.mean(X > 1), np.var(X)

In [16]:
# Variance reduction by conditioning: Calculate P(X > 1 | Y)
def mc_cond(M, U1):
    Y = -np.log(1 - U1)  # exp(1) distribution from U(0,1)
    P_cond = 1 - norm.cdf((1 - Y) / np.sqrt(4))  # Conditional P(X > 1 | Y)
    return np.mean(P_cond), np.var(P_cond)

In [17]:
# Antithetic variates for variance reduction
def mc_anti(M, U1, U2):
    U1 = np.clip(U1, 1e-10, 1 - 1e-10)  # Clip U1 to avoid exact 0 or 1
    Y = -np.log(1 - U1)  # exp(1) distribution from U(0,1)
    X = np.sqrt(4) * U2 + Y  # Gaussian with mean Y and variance 4
    Y_anti = -np.log(1 - (1 - U1))  # Antithetic for Y
    X_anti = np.sqrt(4) * (-U2) + Y_anti # Antithetic for X
    return np.mean((X > 1) + (X_anti > 1)) / 2, np.var(((X > 1) + (X_anti > 1)) / 2)

In [18]:
# Control variate technique for variance reduction
def mc_control_var(M, U1, U2):
    Y = -np.log(1 - U1)  # Exponential(1) distribution
    X = np.sqrt(4) * U2 + Y  # Gaussian with mean Y and variance 4
    cov_XY = np.cov(X > 1, Y)[0, 1]
    var_Y = np.var(Y)
    a = cov_XY / var_Y  # Optimal control variate coefficient
    return np.mean((X > 1) - a * (Y - 1)), np.var((X > 1) - a * (Y - 1))

In [19]:
# List to Store results
results = []

# Perform simulations for different values of M
for m_ in M:
    # Generate U(0, 1) random numbers using LCG
    U1 = lcg(a, c, m, seed, m_)
    U2 = lcg(a, c, m, seed + 1, m_)
    
    # Basic Monte Carlo estimate
    P_basic, V_basic = mcb(M, U1, U2)
    
    # Variance reduction by conditioning
    P_cond, V_cond = mc_cond(M, U1)
    
    # Antithetic variates estimate
    P_anti, V_anti = mc_anti(M, U1, U2)
    
    # Control variates estimate
    P_control_var, V_control_var = mc_control_var(M, U1, U2)
    
    # Store results
    results.append([m_, P_basic, V_basic, P_cond, V_cond, P_anti, V_anti, P_control_var, V_control_var])

In [20]:
# Creating the table by using dataframe
df = pd.DataFrame(results, columns=['M', 'Basic Estimation', 'Basic Variance', 'Conditional Estimation', 'Conditional Variance', 'Antithetic Estimation', 'Antithetic Variance', 'Control Variate Estimation', 'Control Variate Variance'])
df['% Reduction (Conditional)'] = (1 - df['Conditional Variance'] / df['Basic Variance']) * 100
df['% Reduction (Anti-thetic)'] = (1 - df['Antithetic Variance'] / df['Basic Variance']) * 100
df['% Reduction (Control)'] = (1 - df['Control Variate Variance'] / df['Basic Variance']) * 100

df

,M,Basic Estimation,Basic Variance,Conditional Estimation,Conditional Variance,Antithetic Estimation,Antithetic Variance,Control Variate Estimation,Control Variate Variance,% Reduction (Conditional),% Reduction (Anti-thetic),% Reduction (Control)
0,10,0.80000,0.379545,0.417195,0.005376,0.450000,0.022500,0.980758,0.139458,98.583565,94.071852,63.256623
1,100,0.77000,0.773561,0.441810,0.014880,0.455000,0.020475,0.838065,0.155012,98.076401,97.353149,79.961223
2,1000,0.79900,1.317449,0.487845,0.025950,0.455500,0.020270,0.801626,0.140256,98.030252,98.461440,89.354001
3,10000,0.81460,1.268191,0.485227,0.025778,0.462350,0.017407,0.818807,0.133723,97.967333,98.627378,89.455605
4,100000,0.81623,1.316731,0.489404,0.026397,0.463195,0.017048,0.816935,0.132568,97.995226,98.705286,89.932028


### Q.3

In [21]:
# Weibull distribution sampling using LCG for uniform random numbers
def weibull(k, sigma, U):
    return sigma * (-np.log(1 - U))**(1 / k)

In [90]:
# Basic Monte Carlo simulation using LCG
def mc_weinbull(n, a, c, m, seed, stratified=False):
    prob = [0.19, 0.26, 0.24, 0.17, 0.14]  # P(N=1), P(N=2), etc.
    N = [1, 2, 3, 4, 5]  # Possible number of storms
    k, sigma = 0.8, 3  # Weibull distribution parameters
    rain_thres = 5  # Rainfall threshold for emergency allocation

    # Generate LCG uniform random numbers (2N for safety)
    U = lcg(a, c, m, seed, n * 10)
    below_thres_count = 0
    index = 0

    for _ in range(n):
        U_N = U[index]
        index += 1
        N_ = np.random.choice(N, p=prob)

        # Sample rainfall depths Di for each storm
        total_rain = 0
        for i in range(N_):
            U_Di = U[index]  # Use LCG random number for Weibull
            index += 1
            Di = weibull(k, sigma, U_Di)
            total_rain += Di
        
        # Check if total rainfall is below the threshold
        if total_rain < rain_thres:
            below_thres_count += 1
    
    # Estimate the probability
    probability = below_thres_count / n
    return probability

In [100]:
# Stratified Monte Carlo simulation using LCG
def stratified_mc(N_, a, c, m, seed):
    prob = [0.19, 0.26, 0.24, 0.17, 0.14]  # P(N=1), P(N=2), etc.
    N = [1, 2, 3, 4, 5]  # Possible number of storms
    k, sigma = 0.8, 3  # Weibull distribution parameters
    rain_thres = 5  # Rainfall threshold for emergency allocation

    strata_cnt = 10  # Number of strata
    sps = N_ // strata_cnt  # Equal samples per stratum
    below_thres_cnt = 0

    for strat in range(strata_cnt):
        # Using LCG to generate uniform random numbers within each stratum
        U = lcg(a, c, m, seed, sps * 10)
        index = 0

        for _ in range(sps):
            # Sample number of storms N
            U_N = U[index]
            index += 1
            n = np.random.choice(N, p=prob)

            # Sample rainfall depths Di for each storm
            total_rain = 0
            for i in range(n):
                U_Di = U[index]
                index += 1
                Di = weibull(k, sigma, U_Di)
                total_rain += Di
            
            # Check if total rainfall is below the threshold
            if total_rain < rain_thres:
                below_thres_cnt += 1
    
    # Estimate the probability
    probability = below_thres_cnt / N_
    return probability

In [101]:
# 99% confidence interval
def c_i(prob, n):
    z = 2.576
#     prob = np.clip(prob, 1e-9, 1 - 1e-9)
    error = z * np.sqrt((prob * (1 - prob)) / n)
    return prob - error, prob + error

In [123]:
M = [100, 10000]
for m_ in M:
    print(f"\nRunning Monte Carlo simulation with {m_} samples...")
    prob_mc = mc_weinbull(m_, a, c, m, seed)
    ci_lower, ci_upper = c_i(prob_mc, m_)
    print(f"Estimated Probability: {prob_mc}")
    print(f"99% Confidence Interval: [{ci_lower}, {ci_upper}]")
    
    print(f"\nRunning Stratified Monte Carlo simulation with {m_} samples...")
    prob_strat_mc = stratified_mc(m_, a, c, m, seed)
    ci_strat_lower, ci_strat_upper = c_i(prob_strat_mc, m_)
    print(f"Estimated Probability (Stratified Monte Carlo): {prob_strat_mc}")
    print(f"99% Confidence Interval (Stratified Monte Carlo): [{ci_strat_lower}, {ci_strat_upper}]")


Running Monte Carlo simulation with 100 samples...
Estimated Probability: 0.42
99% Confidence Interval: [0.29285932855297636, 0.5471406714470236]

Running Stratified Monte Carlo simulation with 100 samples...
Estimated Probability (Stratified Monte Carlo): 0.4
99% Confidence Interval (Stratified Monte Carlo): [0.2738022884518107, 0.5261977115481893]

Running Monte Carlo simulation with 10000 samples...
Estimated Probability: 0.3769
99% Confidence Interval: [0.3644164586817336, 0.3893835413182664]

Running Stratified Monte Carlo simulation with 10000 samples...
Estimated Probability (Stratified Monte Carlo): 0.3666
99% Confidence Interval (Stratified Monte Carlo): [0.35418687529247594, 0.379013124707524]
